In [1]:
import torchvision
import torch
import train
from metrics import StreamSegMetrics
from utils import ext_transforms as et
import datasets.acdc
from torch.utils import data
from tqdm import tqdm
from datasets import Cityscapes
import rotation
from torchvision import transforms
import network

In [2]:
device = 'cuda:2' if torch.cuda.is_available else 'cpu'

In [3]:
NUM_CLASSES=19

In [4]:
model = torchvision.models.segmentation.deeplabv3_resnet50(num_classes=NUM_CLASSES)
model = torch.nn.DataParallel(model, device_ids=[2,1])
model.to(device) 
checkpoint = torch.load('model_20230925_152430_200')
model.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [4]:
model = torchvision.models.segmentation.deeplabv3_resnet50(num_classes=NUM_CLASSES)
model = torch.nn.DataParallel(model, device_ids=[2,1])
model.to(device)
checkpoint = torch.load('adapted_models/resnet_50_CS_fog')
model.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [5]:
model = torchvision.models.segmentation.deeplabv3_mobilenet_v3_large(num_classes=19) 
model = torch.nn.DataParallel(model, device_ids=[3,2])
model.to(device)
checkpoint = torch.load('mobilenet_20231130_221541_last')
model.load_state_dict(checkpoint['model'])
#model = torch.nn.DataParallel(model, device_ids=[2,0,1])

<All keys matched successfully>

In [6]:
model

DataParallel(
  (module): DeepLabV3(
    (backbone): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       

In [5]:
train_transform = et.ExtCompose([
    et.ExtRandomScale(scale_range=(0.5, 2.0)),
    et.ExtRandomCrop(size=(512,1024), pad_if_needed=True),
    et.ExtRandomHorizontalFlip(),
    et.ExtToTensor(),
    et.ExtNormalize(mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225])
])

In [6]:
to_tensor = et.ExtCompose([et.ExtToTensor()])
val_transform = et.ExtCompose([
            et.ExtToTensor(),
            et.ExtNormalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225])
])

In [7]:
WEATHER = 'fog'

In [8]:
validation_set = datasets.acdc.Acdc(r"/vol/tmp/phanfran/acdc/", 'val', 'semantic',transform=val_transform, weather=WEATHER)
BATCH_SIZE = 1
val_loader = data.DataLoader(
        validation_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=8,
        drop_last=True)

In [9]:
tensor_set = datasets.acdc.Acdc(r"/vol/tmp/phanfran/acdc/", 'train', 'semantic',transform=to_tensor, weather=WEATHER)
BATCH_SIZE = 1
tensor_loader = data.DataLoader(
        tensor_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=8,
        drop_last=True)

In [8]:
#momentan val
cityscapes_train = Cityscapes(root='/vol/tmp/phanfran/cityscapes/',
                             subset=None,
                             split='train',
                             transform=to_tensor)

BATCH_SIZE = 1
cityscapes_train_loader = data.DataLoader(
        cityscapes_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=2,
        drop_last=True)

cityscapes_to_tensor = Cityscapes(root='/vol/tmp/phanfran/cityscapes/',
                             subset=WEATHER,
                             split= 'train',
                             transform=to_tensor)

BATCH_SIZE = 1
cityscapes_tensor_loader = data.DataLoader(
        cityscapes_to_tensor, batch_size=BATCH_SIZE, shuffle=True, num_workers=2,
        drop_last=True)


cityscapes_val = Cityscapes(root='/vol/tmp/phanfran/cityscapes/',
                             subset=WEATHER,
                             split='val',
                             transform=val_transform)

BATCH_SIZE = 1
cityscapes_val_loader = data.DataLoader(
        cityscapes_val, batch_size=BATCH_SIZE, shuffle=True, num_workers=2,
        drop_last=True)


In [23]:
cs_train_subset = torch.utils.data.random_split(cityscapes_train,[len(tensor_set), len(cityscapes_train)-len(tensor_set)])[0]
dua_dataset = torch.utils.data.ConcatDataset([tensor_set, cs_train_subset])
dua_loader = data.DataLoader(
        dua_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8,
        drop_last=True)
len(dua_loader)

NameError: name 'cityscapes_train' is not defined

In [9]:
NUM_CLASSES = 19
metrics = StreamSegMetrics(n_classes=NUM_CLASSES)
metrics.reset()

In [68]:
train.validate(model,device,val_loader,metrics)

100%|██████████| 100/100 [00:34<00:00,  2.90it/s]


zero shot performance

fog

In [13]:
metrics.get_results()["Mean IoU"]

0.36798932500582315

In [37]:
metrics.get_results()["Overall Acc"]

0.8283137144456527

fog class iou

In [11]:
metrics.get_results()["Class IoU"]

{0: 0.8136252086906332,
 1: 0.5784427004845875,
 2: 0.6039401087070907,
 3: 0.014069694179227451,
 4: 0.14731043394889323,
 5: 0.10838495183238434,
 6: 0.43082521551348374,
 7: 0.3909355395945086,
 8: 0.7645908221623414,
 9: 0.371580624728042,
 10: 0.828291355709795,
 11: 0.046333172307397766,
 12: 0.29940815609545174,
 13: 0.551099930362755,
 14: 0.2756943891807123,
 15: 0.26591734056239663,
 16: 0.458828221677626,
 17: 0.01213401882188916,
 18: 0.03038529055142335}

after adaptation

In [24]:
metrics.get_results()["Class IoU"]

{0: 0.5803105692053315,
 1: 0.7748841049451624,
 2: 0.7221452487345872,
 3: 0.4074058608168425,
 4: 0.34155661660647296,
 5: 0.3290132621893751,
 6: 0.6757933271796894,
 7: 0.6297909407665505,
 8: 0.8445886977384528,
 9: 0.565186441222651,
 10: 0.6527708065089354,
 11: 0.21673594262149673,
 12: 0.4685159580596843,
 13: 0.7859093988476006,
 14: 0.3916949459092428,
 15: 0.4983509539290935,
 16: 0.6821415641776224,
 17: 0.19227771753862835,
 18: 0.32807483862059444}

In [25]:
metrics.get_results()["Overall Acc"]

0.8111054445883801

rain

In [71]:
metrics.get_results()["Mean IoU"]

0.3139140494860081

before adaptation

In [69]:
metrics.get_results()["Overall Acc"]

0.8197065922716312

In [73]:
metrics.get_results()["Class IoU"]

{0: 0.7038052610332651,
 1: 0.17714505359531812,
 2: 0.5826472640480683,
 3: 0.09799081859283247,
 4: 0.0526112965742836,
 5: 0.1833748343664126,
 6: 0.37245427754499116,
 7: 0.4836942928655799,
 8: 0.7960535351071651,
 9: 0.3098140695984662,
 10: 0.9032899226779644,
 11: 0.18651383975427144,
 12: 0.0,
 13: 0.5847077538921891,
 14: 0.11932312708884785,
 15: 0.22593835253491548,
 16: 0.11630257308100878,
 17: 0.05398335851354094,
 18: 0.014717309365033343}

after adaptation

In [56]:
metrics.get_results()["Overall Acc"]

0.88715597230059

In [55]:
metrics.get_results()["Class IoU"]

{0: 0.750300819003998,
 1: 0.426480151109026,
 2: 0.7874959700914547,
 3: 0.26022939957404395,
 4: 0.2734242309260894,
 5: 0.39713846340850967,
 6: 0.6063566348972974,
 7: 0.6749890199548955,
 8: 0.8922349860374934,
 9: 0.5377594455191469,
 10: 0.9326872990168597,
 11: 0.5004070958462276,
 12: 0.03878587375215445,
 13: 0.7829552605719134,
 14: 0.22172218437467553,
 15: 0.2821754410119945,
 16: 0.16052009562353714,
 17: 0.1755419033267648,
 18: 0.3800453852163238}

snow

In [86]:
metrics.get_results()["Mean IoU"]

0.2570405916582452

night

In [13]:
metrics.get_results()["Mean IoU"]

/vol/fob-vol3/mi20/phanfran/Bachelorarbeit/dua_deeplab_tests/metrics/stream_metrics.py:65: RuntimeWarning: invalid value encountered in divide
  acc_cls = np.diag(hist) / hist.sum(axis=1)


0.0796316777263879

cityscapes fog

In [18]:
metrics.get_results()["Mean IoU"] #severity 0.02

0.5287209149819646

In [9]:
metrics.get_results()["Mean IoU"] #severity 0.01

0.6367048012426096

In [14]:
metrics.get_results()["Mean IoU"] #severity 0.005

0.6914316670628892

cityscapes rain

In [12]:
metrics.get_results()["Mean IoU"]

/vol/fob-vol3/mi20/phanfran/Bachelorarbeit/dua_deeplab_tests/metrics/stream_metrics.py:65: RuntimeWarning: invalid value encountered in divide
  acc_cls = np.diag(hist) / hist.sum(axis=1)


0.5097203152300265

mobileNet

vainf params cs

In [ ]:
metrics.get_results()["Mean IoU"]

In [18]:
metrics.get_results()["Mean IoU"]

0.568577832964277

CS

In [18]:
metrics.get_results()["Mean IoU"]  #newest lr 0.01 cross entropy 1000 iters

0.5598166049241478

In [19]:
metrics.get_results()["Mean IoU"]  #newest lr 0.01 cross entropy #Wrong

0.5598166049241478

In [38]:
metrics.get_results()["Mean IoU"] 

0.5054556080669917

In [17]:
metrics.get_results()["Mean IoU"] # focal loss adjusted lr

0.5340697698704586

In [16]:
metrics.get_results()["Mean IoU"]

0.5014643712101446

acdc fog

In [15]:
metrics.get_results()["Mean IoU"]

0.1698328765206157

In [66]:
metrics.get_results()["Mean IoU"]

0.22358372536298143

acdc rain

In [52]:
metrics.get_results()["Mean IoU"]

0.21383255046953673

In [47]:
metrics.get_results()["Mean IoU"] #focal loss

0.19951423009704491

In [10]:
tr_transform_adapt = transforms.Compose([
            transforms.ToPILImage(),
            transforms.RandomCrop((512,1024)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

In [11]:
NUM_SAMPLES = 80

In [12]:
NR_AUGMENTS = 8

In [13]:
def get_adaption_inputs(img, tr_transform_adapt, device):
    img = img.squeeze(0)
    inputs = [(tr_transform_adapt(img)) for _ in range(NR_AUGMENTS)]
    inputs = torch.stack(inputs)
    #inputs_ssh, _ = rotation.rotate_batch(inputs, 'rand')
    #inputs_ssh = inputs_ssh.to(device, non_blocking=True)
    #inputs_ssh /= 255
    return inputs

In [14]:
mom_pre = 0.1
DECAY_FACTOR = 0.94
MIN_MOMENTUM_CONSTANT = 0.005

model.to(device)
model.eval()
results = []
train.validate(model,device,cityscapes_val_loader,metrics)
results.append(metrics.get_results()["Mean IoU"])

with torch.no_grad():
    for i in tqdm(range(NUM_SAMPLES)):
        image, _ = next(iter(cityscapes_tensor_loader))
        mom_new = (mom_pre * DECAY_FACTOR)
        #for m in model.module.backbone.modules():
        for m in model.modules():
            if isinstance(m, torch.nn.modules.batchnorm._BatchNorm):
                m.momentum = mom_new + MIN_MOMENTUM_CONSTANT
                m.train()

        mom_pre = mom_new

        #augment
        inputs = get_adaption_inputs(image, tr_transform_adapt, device)

        #forward pass
        _ = model(inputs)

        #statistics
        train.validate(model,device,cityscapes_val_loader,metrics)
        results.append(metrics.get_results()["Mean IoU"])
        print(metrics.get_results()["Mean IoU"])

  0%|          | 0/500 [00:00<?, ?it/s]

  1%|▏         | 1/80 [01:15<1:39:21, 75.46s/it]

0.5286895430738616


  2%|▎         | 2/80 [02:25<1:34:07, 72.40s/it]

0.5489159101802263


...

0.6001814137421729


 99%|█████████▉| 79/80 [1:53:06<02:04, 124.64s/it]

0.6001732361855547


100%|██████████| 80/80 [1:55:10<00:00, 86.39s/it] 

0.6003586886512678


saving


In [15]:
model_path = 'adapted_models/resnet_50_CS_fog'
torch.save({'model': model.state_dict()}, model_path)

mobile net rain focal loss

In [53]:
results

[0.19951423009704491,
 0.23522699277815864,
 0.25380926085806843,
 0.2687091107079926,
 0.27989655787738266,
 0.2923545404633591,
 0.3000074617340478,
 0.30918932095442,
 0.31429815182929516,
 0.320441039411892,
 0.32507689326194683,
 0.32531040150184043,
 0.3252074074445163,
 0.3266964552721263,
 0.3280010392202381,
 0.3277242165535378,
 0.32925083416519957,
 0.3318191764417547,
 0.3332569711346607,
 0.33162890174693915,
 0.332611712082106,
 0.3345754192681956,
 0.3334261256770934,
 0.3325838284796621,
 0.332647139925351,
 0.3332870321741115,
 0.33134553999571853,
 0.3310127189692221,
 0.3307834886803408,
 0.3297981751384663,
 0.33065654045976645,
 0.32963419879692557,
 0.329413571497935,
 0.32860112260071,
 0.33012829982675357,
 0.3295365261816002,
 0.3298136525351933,
 0.32923801581524925,
 0.32878208885716925,
 0.32941048387022065,
 0.32986578090248636,
 0.33016835385908755,
 0.33005005614292743,
 0.3300979464683853,
 0.3300457182067725,
 0.32928090495255874,
 0.3292742320031036,
 

mobile net fog focal loss

In [82]:
results

[0.22358372536298143,
 0.2554378937152351,
 0.2908849125351911,
 0.31681900210139613,
 0.3314999751453472,
 0.3428528100479516,
 0.34810161882070145,
 0.353552161114579,
 0.3606744336539561,
 0.36231737296149696,
 0.3660556119117553,
 0.3678984355220381,
 0.3700758282218907,
 0.3708945820467419,
 0.3701942251889682,
 0.3674957864532291,
 0.36667851249253863,
 0.3642780469244458,
 0.36518179609963813,
 0.36433573180413453,
 0.3630993912712861,
 0.36185141210886174,
 0.36534408527223095,
 0.36394573960456694,
 0.363572100114482,
 0.3639311640975214,
 0.3646962867962156,
 0.3644658339198602,
 0.36073201140357175,
 0.3582342494239001,
 0.35822114739361166,
 0.35860397910473746,
 0.3586387705048107,
 0.35831887906321314,
 0.3590781583175151,
 0.35836455034087206,
 0.3576741011516172,
 0.35749052511961094,
 0.35714259919410235,
 0.3572802633252643,
 0.35623684368208763,
 0.3564745705031071,
 0.35534985944970177,
 0.3552358250563789,
 0.3547191774334377,
 0.35369989733558743,
 0.3545852596248

In [23]:
results

[0.22358372536298143,
 0.255571726563132,
 0.2914731239787449,
 0.3113094434490527,
 0.33283440360079125,
 0.35369100063772263,
 0.3641508360883164,
 0.3700525314756383,
 0.3729152437984948,
 0.37824277389494143,
 0.3817205452910861,
 0.3808261741288856,
 0.38026887291535116,
 0.3803320999285288,
 0.382303938346562,
 0.381119351974379,
 0.37969153931733635,
 0.3786326648214809,
 0.3778919120743261,
 0.37847382094493753,
 0.37802265143763547,
 0.37570393672083524,
 0.37274304006638476,
 0.3724332707601714,
 0.37216082948194973,
 0.3712055363023832,
 0.3711365582656662,
 0.3721236091509538,
 0.3732502284662912,
 0.37289356664342727,
 0.3736549411532451,
 0.3717178818137034,
 0.37083275967656715,
 0.37099226782068345,
 0.3702662276638212,
 0.3690057019964491,
 0.368602983708998,
 0.36749495994415904,
 0.3662943702453683,
 0.36594622688801204,
 0.36641059571477613,
 0.3660897404271282,
 0.3652156336004677,
 0.36632691927097555,
 0.3658250573391955,
 0.3657684070311713,
 0.36627506354919426

mobile net rain

In [62]:
results

[0.19889463147238629,
 0.21651021684522287,
 0.2508395785717952,
 0.2576400447078906,
 0.264167524265594,
 0.274134399540847,
 0.2822033188825226,
 0.28812741812777576,
 0.2897983841260904,
 0.2885543999230986,
 0.29203099963403784,
 0.2924041766751749,
 0.29417467302586303,
 0.2952592960671983,
 0.2954826164622149,
 0.29810419539860455,
 0.2992247314254184,
 0.2985998646483467,
 0.29943306975343326,
 0.2979808589448507,
 0.2987282807460354,
 0.2995347313861746,
 0.29992399062935454,
 0.30087075191790796,
 0.30102693995378454,
 0.30190488704728374,
 0.3016234940442724,
 0.3009109519966919,
 0.30128438699999616,
 0.3005464280577663,
 0.3011602662689332,
 0.30129735572589317,
 0.30165517542269266,
 0.3021153323937305,
 0.3023503991744141,
 0.30271059466806105,
 0.3031066094673332,
 0.3033501057073701,
 0.30326111989639787,
 0.30296139548726153,
 0.30239755239792687,
 0.30273218282410286,
 0.3031471724120572,
 0.30327617421887926,
 0.3032268038047764,
 0.30339220645889486,
 0.303359316526

mobile net fog

In [44]:
results

[0.17434194327232985,
 0.19875614027008898,
 0.22027968675018814,
 0.23552427709382137,
 0.26091461611500694,
 0.274242899279018,
 0.28257540392476893,
 0.29179575332904195,
 0.29563470285211413,
 0.3033447708312181,
 0.305427580446828,
 0.306525147139888,
 0.3089524333259205,
 0.31137863991451886,
 0.3081828228886667,
 0.30725538143726144,
 0.3018740900711873,
 0.2991844511414681,
 0.2964477382931025,
 0.2944866649174941,
 0.2951557327015818,
 0.2986029245525537,
 0.3000910934249031,
 0.303096283112472,
 0.305627756778891,
 0.30757131196940024,
 0.3068133638222854,
 0.30639394982307167,
 0.30620116002241954,
 0.3057112688632948,
 0.3054992595677206,
 0.3072331706064141,
 0.3061482287093166,
 0.30624504622499427,
 0.30496207963708494,
 0.30457012529662175,
 0.3045381290245386,
 0.30455223896768086,
 0.30625175149826245,
 0.3063510233026869,
 0.30507390944899015,
 0.3038876015424751,
 0.3046054057925276,
 0.30444243865372833,
 0.30413848686277495,
 0.30398653103691337,
 0.30400869446702

acdc rain relay (%2)

In [ ]:
results

acdc fog relay (%2)

In [19]:
results

[0.36798932500582315,
 0.42175101159644496,
 0.42980128996853706,
 0.4527815553858991,
 0.4521972788221678,
 0.47603138153803637,
 0.4684861102983221,
 0.4855632830589292,
 0.48504826923904487,
 0.49427448105160204,
 0.48786190855375694,
 0.4968522391924309,
 0.49409638234148356,
 0.503034675726308,
 0.4983076031943689,
 0.5047989386257623,
 0.5005067451989129,
 0.5057988832621004,
 0.5012403702952287,
 0.5069874635309976,
 0.503887094959746,
 0.5055623833989252,
 0.5028739118408166,
 0.506682264705918,
 0.5036207216477994,
 0.5066999727144587,
 0.5066415335893367,
 0.5142595806484345,
 0.5111760866402696,
 0.51375612304722,
 0.5117595113350323,
 0.5134768915897646,
 0.5121353855208148,
 0.5161901968192635,
 0.514851676602833,
 0.5165556643026941,
 0.5135102200460442,
 0.5147392381172602,
 0.5151665971570331,
 0.5186788683711986,
 0.5171603589575495,
 0.5190948984096644,
 0.5168706480930172,
 0.5187921271471757,
 0.5177091599234069,
 0.5196302998638067,
 0.519484710194408,
 0.521387868

cs fog (severity 2.0)

In [24]:
results

[0.36798932500582315,
 0.5416354635839227,
 0.559191534051159,
 0.573505867426431,
 0.5829954125529643,
 0.5887437561704387,
 0.592890826597204,
 0.5970800279199096,
 0.59994018561926,
 0.6053699024669851,
 0.6046294702076834,
 0.6030094398390746,
 0.6005188115446982,
 0.6004453397369978,
 0.6025324282199972,
 0.6013771068202032,
 0.6014274785706503,
 0.5999247915612128,
 0.5989491299344195,
 0.5975427385852476,
 0.5960945672028745,
 0.5966385993993931,
 0.5959074901959176,
 0.5955064018991558,
 0.5958157487977414,
 0.5949714987457224,
 0.5949830292723327,
 0.5950359186925579,
 0.5964464092053916,
 0.5971843791452972,
 0.5968484775895767,
 0.5965717356406796,
 0.5962224664172995,
 0.596163324252337,
 0.5959829492899796,
 0.5966418214896554,
 0.5964248280805307,
 0.5965013374524378,
 0.5958742404262725,
 0.5971810941864131,
 0.5966294016202529,
 0.5966810641683132,
 0.5966006955285084,
 0.5971883063689594,
 0.5973227869117159,
 0.5971715954674973,
 0.5972592780084894,
 0.597034658777623

acdc fog backbone only

In [21]:
results

[0.36798932500582315,
 0.41914032757073477,
 0.4574474262739763,
 0.48915433717768986,
 0.5033067117402376,
 0.5189854546080648,
 0.5306392277187078,
 0.5398955466016343,
 0.5475201337600687,
 0.554330771676177,
 0.5559993295701005,
 0.5627267321394144,
 0.5693741933691738,
 0.5703775664668155,
 0.5691246769560665,
 0.5708849148781715,
 0.5668200258173886,
 0.5653332541726831,
 0.5691173796958872,
 0.56540002181323,
 0.5670885122587057,
 0.5656262142293591,
 0.5671485207468363,
 0.5669904432474003,
 0.568295570202955,
 0.5683429981011372,
 0.5681456191724511,
 0.5689638114603908,
 0.5705298931033179,
 0.5699451167046589,
 0.5708692423093122,
 0.5687034624702756,
 0.5689358302233888,
 0.5696852364973083,
 0.5692265730459021,
 0.5670312131465006,
 0.5680465730328875,
 0.5675230959396768,
 0.5664826071988024,
 0.5664024895029479,
 0.5656748765512126,
 0.5656969333237915,
 0.5659508120690034,
 0.5656099061971119,
 0.5646737091715658,
 0.5656689958347397,
 0.5655470384059764,
 0.56678208807

acdc relay of infos

In [20]:
results

[0.36798932500582315,
 0.42423648101101696,
 0.4566834948701865,
 0.476433050423001,
 0.48831899071338486,
 0.5003045188499111,
 0.5099149155028597,
 0.5227463546051669,
 0.5240482427412558,
 0.5256679741266012,
 0.5260328194171852,
 0.5265954481305231,
 0.5265932825224718,
 0.5258231825992865,
 0.5313404340783892,
 0.5323224355087429,
 0.535480558862012,
 0.537657487834587,
 0.5368655143788991,
 0.5371414830786342,
 0.5387499227009815,
 0.537342182289139,
 0.5381955104994925,
 0.5347597497255759,
 0.5376210177166338,
 0.5407426787900148,
 0.5397944807895804,
 0.5364625631045566,
 0.5368368586080189,
 0.5343904958054317,
 0.535309728398094,
 0.5343235815724731,
 0.5329274499267891,
 0.5336553885437973,
 0.5351172345527367,
 0.5342265427144944,
 0.5351934012790655,
 0.53404466511616,
 0.5325797670389772,
 0.5313753008419763,
 0.5330545850611834,
 0.533190355371844,
 0.5314543009852649,
 0.5310752698754355,
 0.5304484159934936,
 0.5309198022020807,
 0.5317608967124454,
 0.532354566239974

acdc fog (decay 0.98, 0.05)

In [18]:
results

[0.36798932500582315,
 0.4421834133909748,
 0.4742474500448582,
 0.4881774414244721,
 0.5114598505715965,
 0.5150079864923706,
 0.5080443291881394,
 0.5185298488085429,
 0.5153481519781087,
 0.5174066190768689,
 0.4983325664731297,
 0.49343430957057405,
 0.4971502443768872,
 0.4801830834623283,
 0.47616924324319637,
 0.4716220964016205,
 0.4701647322814988,
 0.4714435556803509,
 0.49382638738159584,
 0.5024852649142807,
 0.5055095523593013,
 0.49393521371475463,
 0.4840776824323788,
 0.5000864461921865,
 0.504238906445544,
 0.4904163616501433,
 0.5008252936161682,
 0.48356077173058587,
 0.5013181041387371,
 0.511219412062763,
 0.511710774841411,
 0.4960388590455982,
 0.4893223670680278,
 0.5030364510681608,
 0.5260128134833155,
 0.5349531046190115,
 0.527223248457117,
 0.5240153531294253,
 0.5105462406107805,
 0.4980353219388417,
 0.513260476464213,
 0.4982613103398453,
 0.4936298762000298,
 0.49621276292223304,
 0.5059584172860403,
 0.5129808131708992,
 0.4995284282269672,
 0.50806323

acdc fog (decay 0.98)

In [18]:
results

[0.36798932500582315,
 0.40717610207373617,
 0.4507858744254784,
 0.46641329064535175,
 0.4780895629927954,
 0.4918317352753814,
 0.4941078615623116,
 0.49846453299470556,
 0.49755935186555544,
 0.5035458454549171,
 0.5108850320252422,
 0.5077293942912185,
 0.5041176416806316,
 0.5197442667020719,
 0.5099394801926731,
 0.5123784290189729,
 0.5165257282050798,
 0.5274441725030211,
 0.5224192143916455,
 0.5323705428066963,
 0.5293202407322368,
 0.5420963820080467,
 0.5502806872685869,
 0.5450541058235074,
 0.533335779432762,
 0.5413462714590274,
 0.5430305676371054,
 0.5340033176288432,
 0.5215096889222101,
 0.5130689246369534,
 0.5102883255616826,
 0.509323369574284,
 0.5041110496794992,
 0.5148075222458516,
 0.5234591906023418,
 0.5163737246475124,
 0.5114866021304482,
 0.5118049873156668,
 0.5080802312815277,
 0.5025887721280288,
 0.49725989692037326,
 0.5011691856624704,
 0.5140604823698292,
 0.5103404219592828,
 0.5073347989019362,
 0.5023424511747007,
 0.5124797536096534,
 0.509312

acdc rain (decay 0.98)

In [18]:
results

[0.3139140494860081,
 0.34568946683885277,
 0.38190570617992203,
 0.4126439353281127,
 0.4314131286463451,
 0.4413925636372108,
 0.447568301578771,
 0.454295057175927,
 0.4602082845808848,
 0.4643007428738651,
 0.4675724739624481,
 0.4708699444896636,
 0.4703865774374794,
 0.4726931293641532,
 0.473793743421719,
 0.47214596358068456,
 0.47303502188599156,
 0.47224328817731637,
 0.468389018296039,
 0.46556872663531873,
 0.46387469587129165,
 0.4600098954941434,
 0.46131573126122677,
 0.46120968556222514,
 0.45789016829955753,
 0.45719056297353977,
 0.45432074438430475,
 0.45781262123887845,
 0.45967625363415066,
 0.4633673235697847,
 0.4605851264339449,
 0.4638804350752313,
 0.4663520304892172,
 0.46342079926918833,
 0.4586686730796299,
 0.4625095445861178,
 0.46486677031724627,
 0.4658654175495269,
 0.46837099812767724,
 0.4679518430591361,
 0.46844946483822997,
 0.4710953650350507,
 0.4715962419328945,
 0.4695186271784317,
 0.46656398620693484,
 0.46414079487517756,
 0.462318894270485

acdc rain (original params)

In [36]:
results

[0.3139140494860081,
 0.34889566499909924,
 0.3764661863357306,
 0.4019331195670249,
 0.4212200848652106,
 0.4348649265445312,
 0.43751349168444725,
 0.4432293794941365,
 0.4485563657849249,
 0.46556809010959743,
 0.4686729376202257,
 0.4722418948210731,
 0.47696451296334147,
 0.4783651734679201,
 0.4777288174820573,
 0.4817410052864711,
 0.4812193744705295,
 0.483189611881795,
 0.48382494257456454,
 0.4839389225448034,
 0.48271573651510485,
 0.4834489907986848,
 0.48238881565153824,
 0.481422106734752,
 0.4813458926803285,
 0.4814948940074545,
 0.48081367716570816,
 0.48124574846450513,
 0.4821754379577119,
 0.4827176430722984,
 0.48207816496084754,
 0.4829540783375787,
 0.48207567423736997,
 0.4814966142036976,
 0.4814479298072758,
 0.4806267985829676,
 0.4808294254788308,
 0.481070656288585,
 0.48165427918748877,
 0.48142800655945833,
 0.4816304854425232,
 0.48053532113888037,
 0.48097458088712325,
 0.4815629193908624,
 0.48085054930653826,
 0.48128578573452674,
 0.4812935865801177,

acdc fog (now with original params)

In [19]:
results

[0.36798932500582315,
 0.4055044220083661,
 0.4493491772856589,
 0.4664402782402463,
 0.4904630391682496,
 0.49670478378691607,
 0.5136583846740667,
 0.524077107335853,
 0.5286650043323854,
 0.5280608570835604,
 0.5341665049820777,
 0.5414005666410135,
 0.542037106369829,
 0.5447008920902299,
 0.5449278044091634,
 0.5452730868245371,
 0.5434218199739052,
 0.5434580460815809,
 0.5415663018789199,
 0.5380891599484968,
 0.541232801964672,
 0.5384162563344244,
 0.5400262073893194,
 0.5425302452306889,
 0.5443046796682809,
 0.5411493920034128,
 0.5418375418488732,
 0.5384438528146772,
 0.5377587937506008,
 0.5387901102040454,
 0.5382015267757371,
 0.5356578337402966,
 0.5328691851500917,
 0.5331621235336695,
 0.5353901098217955,
 0.5358803948785282,
 0.5356235929432864,
 0.533690755492378,
 0.5335263357633785,
 0.5331728345599857,
 0.5348178075121912,
 0.533377495082263,
 0.535868778008546,
 0.5370063454148376,
 0.5379418037259064,
 0.5364673430121791,
 0.5354086783780613,
 0.53704227070208

acdc rain (adaptation on train set)

In [19]:
results

[0.34992054570749864,
 0.3825789910903077,
 0.40141714733979444,
 0.41304840527264386,
 0.4276511925792971,
 0.4333550653181568,
 0.4425395845121709,
 0.44947619860547827,
 0.455074553651247,
 0.4609184242817854,
 0.4704279287831044,
 0.47240981301929547,
 0.47277587680977523,
 0.4745405106590032,
 0.47518554204984315,
 0.4764246434586073,
 0.4810529727318013,
 0.4788680742678728,
 0.4812041827235574,
 0.482694139584899,
 0.482262496266636,
 0.4836030996193864,
 0.4830634050648571,
 0.4831182310477537,
 0.48442360168111626,
 0.48344305948379945,
 0.48269482897293187,
 0.4834138090337085,
 0.4845371630564952,
 0.48327098459438006,
 0.4848445370030975,
 0.4848251614406961,
 0.4838683148687702,
 0.4841974054977359,
 0.4838203304256838,
 0.4839738373073285,
 0.48447314892783844,
 0.4838333043149312,
 0.4835020077009312,
 0.48259026631459423,
 0.48310078169077697,
 0.4831369865610659,
 0.48276127439687855,
 0.48264594919948556,
 0.483243094007337,
 0.48304483157354383,
 0.4832554499326722,


2.testrun

In [16]:
results

[0.3399669862499999,
 0.3708504498837393,
 0.4154070402148753,
 0.4294493852682889,
 0.4379762575004083,
 0.45167127923684147,
 0.45507185478756457,
 0.4606166772534046,
 0.46345134904493623,
 0.46654373641575964,
 0.470353065774906,
 0.4723590465279658,
 0.47340037003732804,
 0.4764796102118512,
 0.4762432194483633,
 0.47535694465877326,
 0.475883863431412,
 0.477428111342586,
 0.4751724241985721,
 0.47538112461853155,
 0.4756848070163806,
 0.47668103290286173,
 0.4773330855670535,
 0.4763176815080672,
 0.4746014583451097,
 0.47526750114920663,
 0.4753409777153133,
 0.4749469669481652,
 0.47541760993789894,
 0.4750354513919084,
 0.47506206681121627,
 0.47468534347804514,
 0.4752641851429524,
 0.4747873026208467,
 0.47577009959246586,
 0.4761901752499194,
 0.475435126287254,
 0.4755164591559602,
 0.4756885082306881,
 0.4749081909919541,
 0.4744084739215409,
 0.47505142402417605,
 0.4752686500902343,
 0.47531470962570854,
 0.4750847180925856,
 0.47382150768995995,
 0.47358461826742976,


acdc rain backbone only

In [17]:
results

[0.36287334019697687,
 0.3819199953724538,
 0.4034312655461667,
 0.4237906075061121,
 0.4365813186149669,
 0.44349766934476126,
 0.4505042964038954,
 0.4604400009026514,
 0.4643496220963387,
 0.4678617781700541,
 0.46808223606559984,
 0.4687163686520151,
 0.4654062921527496,
 0.46760348359735354,
 0.4672015471644278,
 0.4709273445079209,
 0.4709134294696491,
 0.47164782024785906,
 0.47183590982683216,
 0.47083795578772186,
 0.4706741348944993,
 0.47010230482640974,
 0.46947905233769427,
 0.47009927466107604,
 0.47006929562611094,
 0.4701338441523467,
 0.46912807768943215,
 0.46961793248021333,
 0.46931538340976214,
 0.4697867694824419,
 0.4703684017737018,
 0.47090280777209814,
 0.47205004313513355,
 0.4715416677134609,
 0.47138657043765453,
 0.47133766631195656,
 0.4719079517750922,
 0.47105281319385206,
 0.471051426793041,
 0.47143445151321445,
 0.4712742142481384,
 0.47088857829595954,
 0.4711999331069768,
 0.471746812178716,
 0.4713991558531397,
 0.4711038568692488,
 0.471875477248

acdc rain, adaptation and validation on val set

In [16]:
results

[0.34589268293955877,
 0.3760385837776261,
 0.4027452938915229,
 0.41691459390568886,
 0.4299512429435893,
 0.43491428351281874,
 0.4399237570639392,
 0.4425655299316052,
 0.4485357936326396,
 0.4513063728226124,
 0.4561659195194859,
 0.4551692337080504,
 0.45570312665471163,
 0.4606297167970872,
 0.462920107500661,
 0.4628393403508437,
 0.46380655844782864,
 0.46305647685597534,
 0.4641636395425542,
 0.4618119432964844,
 0.4611464461395764,
 0.46062115613347937,
 0.4595827607669213,
 0.461732587991208,
 0.4594189852862199,
 0.4578746705287944,
 0.4580917125482996,
 0.45635949312162466,
 0.45667157746831405,
 0.4550885810495058,
 0.45498957964252434,
 0.45316831816163716,
 0.45202438079990404,
 0.4525415453121128,
 0.4529277949095749,
 0.45409446207955356,
 0.4535362287848049,
 0.45234616350509116,
 0.4523257044204504,
 0.4514481144486509,
 0.4522699886885653,
 0.4528779323376783,
 0.452069401315702,
 0.45237392472888527,
 0.4525658898111304,
 0.4526305408365679,
 0.4537250663499874,
 

In [15]:
results

[0.3432499257539529,
 0.3719318727334886,
 0.39767028579572317,
 0.41306827556493936,
 0.427049652918232,
 0.4367183627748834,
 0.43939056734431786,
 0.4459270988635629,
 0.44852959758740146,
 0.4477762174787166,
 0.44927457792152053,
 0.44929341458786365,
 0.4517245977415795,
 0.45346355127814847,
 0.4512473961942588,
 0.4474734138479978,
 0.4514705053950651,
 0.44867084532194923,
 0.45060541180257085,
 0.4532175088684704,
 0.44991721579355265,
 0.4507419908945833,
 0.4487084844923634,
 0.4463943244584231,
 0.44506629927589864,
 0.44448014935694186,
 0.446927244871074,
 0.44620888639062406,
 0.44789128899780384,
 0.44626405811431785,
 0.44507359226715326,
 0.4465298238084831,
 0.4473995462656344,
 0.44737219969708863,
 0.4463401368970542,
 0.44592381980673895,
 0.44628161558831414,
 0.4465741044265059,
 0.4478020043981653,
 0.44934747680318354,
 0.44913298057311385,
 0.44883291293862304,
 0.4487806543672859,
 0.4494418705413232,
 0.44916376932640184,
 0.44935933370382813,
 0.449608605

In [15]:
results

[0.3538719028949701,
 0.37738609459201916,
 0.3933213034922456,
 0.41739722903519616,
 0.43336413133924223,
 0.44418448856424675,
 0.44579086549305846,
 0.44737746624648606,
 0.45220465651534264,
 0.4551456742229332,
 0.4580085931293883,
 0.45852196946167456,
 0.4614111222096846,
 0.46084422642995304,
 0.4612411282189834,
 0.46376250314428336,
 0.46310151785415615,
 0.46367923966843033,
 0.46123004307494386,
 0.45928567280593013,
 0.45767012987861766,
 0.4595772906047937,
 0.4610583864467732,
 0.4596378541750069,
 0.4584013926966132,
 0.45916408288931965,
 0.45799900032273366,
 0.4592155211867983,
 0.4582648547127694,
 0.45904909954503603,
 0.45903834228874985,
 0.4593717414872558,
 0.46123432295413097,
 0.4599559344330137,
 0.46043667056529197,
 0.4615657441907873,
 0.46053834533412596,
 0.4608858656209016,
 0.45960366187879886,
 0.45847754173047833,
 0.4589914337811838,
 0.45850677727655925,
 0.4596851322066398,
 0.45909668780733937,
 0.4581212734376063,
 0.4576159991716446,
 0.45748

acdc fog

In [16]:
results

[0.4179259660020968,
 0.4595477853238565,
 0.4798331002997702,
 0.4930279270169052,
 0.5105715112772993,
 0.5216695348955053,
 0.5283505080991716,
 0.5343859444385002,
 0.5397564400592458,
 0.5404476018671639,
 0.5434406066537409,
 0.5436232817317146,
 0.5458505166932701,
 0.5440105746913899,
 0.5421782847628006,
 0.5411579076379817,
 0.5448121168525969,
 0.5455106198567788,
 0.5460129571644381,
 0.5446360087180434,
 0.547168419613405,
 0.5504406539839694,
 0.5482765705212816,
 0.5457886804301602,
 0.5430757773177712,
 0.5396689697176794,
 0.5393500977676485,
 0.5385394094855921,
 0.5363270963056783,
 0.5349822535313672,
 0.5340217049372553,
 0.5322243969308181,
 0.5305949088313714,
 0.5322114514739468,
 0.5337265590689849,
 0.5356527208207538,
 0.5345061501205524,
 0.5354470447696511,
 0.5361098033323725,
 0.5380420221154378,
 0.5364692985239653,
 0.5360250177765371,
 0.5361615775022215,
 0.5372570759626157,
 0.5363648198288126,
 0.5350931692384749,
 0.5366101061526726,
 0.53933271090

In [17]:
results

[0.40366273376026046,
 0.45498634727319265,
 0.4876196450401517,
 0.49827970281426137,
 0.5028277914104953,
 0.5105743689629115,
 0.5162051746369327,
 0.5178956595849075,
 0.5220297616055817,
 0.5332046594787814,
 0.5357598413636778,
 0.5440464035969775,
 0.5511246637056098,
 0.5500267554928504,
 0.5504965771838175,
 0.5496094690069097,
 0.5553518024147736,
 0.5553064620879878,
 0.5548829639250141,
 0.5533087751843278,
 0.5483849161505169,
 0.5481701072870028,
 0.5454959308248034,
 0.5421471286029756,
 0.541321421925301,
 0.5401337705482425,
 0.5398874702416379,
 0.5393729021905304,
 0.53713389122389,
 0.5370976779148918,
 0.5374716713115391,
 0.5359368676584582,
 0.5348946061888115,
 0.5374501398261733,
 0.5359135322926634,
 0.5354362581984785,
 0.5342662361786218,
 0.5327675672668724,
 0.5308332244447034,
 0.5320125399222057,
 0.5320750273514342,
 0.5298269209995643,
 0.5297322029503345,
 0.5278852183867516,
 0.5292243271417436,
 0.5271912723548531,
 0.5266647193542461,
 0.5284312953

acdc fog backbone only

In [16]:
results

[0.42880348465839424,
 0.4600999589189058,
 0.47794175751601137,
 0.48879282522257644,
 0.5116729708839318,
 0.5278547709939149,
 0.5315847978583579,
 0.5361419650954897,
 0.5393384491554783,
 0.5360484129502252,
 0.5438893113807822,
 0.5435251671249093,
 0.5451804410836679,
 0.549203127657574,
 0.5466154998883366,
 0.5434195755834812,
 0.5485542536519952,
 0.5506966942925188,
 0.5469906872235066,
 0.5450231322831247,
 0.5445099491870973,
 0.5436141208792783,
 0.5456647821601034,
 0.5456067005162888,
 0.5435707454280183,
 0.5475508318365646,
 0.5497605522933593,
 0.549091850808686,
 0.5480041127060878,
 0.54635304703937,
 0.5480144001192665,
 0.5502687815362087,
 0.5499261090183638,
 0.5501213005493782,
 0.5495594229803565,
 0.5529048768254107,
 0.55184312053884,
 0.5528363656140692,
 0.5528847589950843,
 0.5535708798084291,
 0.5553997493962706,
 0.5543223519183603,
 0.5555971535485861,
 0.5558886963634763,
 0.5574708204562189,
 0.5585117563270793,
 0.5591699806294405,
 0.5574484229402

acdc fog adaptation and validation on val set

In [16]:
results

[0.4217933112317977,
 0.46524820934034206,
 0.4912217434591814,
 0.5027961084716165,
 0.5155598293102625,
 0.5229608249443372,
 0.5272043497511321,
 0.5326661689232676,
 0.541781534976086,
 0.5435440667741297,
 0.5380014964764964,
 0.5406361513333598,
 0.5400130292191507,
 0.547783420106526,
 0.5475542813934962,
 0.5471766436765666,
 0.550594541703768,
 0.548905203467531,
 0.545232762725952,
 0.5486011566332202,
 0.5522036434636108,
 0.5549303118547032,
 0.55655157862112,
 0.557613186638025,
 0.5586360372901918,
 0.5595855766583526,
 0.5620671924013295,
 0.5646233559729876,
 0.5645702673728024,
 0.5616715221559062,
 0.559298116825359,
 0.5617002784506676,
 0.5600373126211582,
 0.5587004450953771,
 0.5601568374570691,
 0.5599626246220445,
 0.5589786976595008,
 0.5590243668697059,
 0.5580440414094971,
 0.559020488296464,
 0.5595531558626234,
 0.5581237491897934,
 0.5563100672323703,
 0.5557167296457424,
 0.5545310515673979,
 0.5558865250622544,
 0.554203483789761,
 0.5522152560204482,
 0

cityscapes fog train adaptation

In [16]:
results

[0.6481392202339523,
 0.6587200715412002,
 0.6567550728484072,
 0.6626212196361815,
 0.6656450635872477,
 0.6664514863334438,
 0.6672239209876905,
 0.6676128898012856,
 0.6681204599971666,
 0.668659305812259,
 0.6683618804786404,
 0.6674184415709441,
 0.6655784012471536,
 0.6629228686475791,
 0.662580612456917,
 0.6632397209250653,
 0.6638136178765043,
 0.667477743319842,
 0.6651720788558948,
 0.6657186610721443,
 0.6621586600354802,
 0.6625498828788409,
 0.6627196731842938,
 0.6622501929607776,
 0.661860584503466,
 0.6620816225425006,
 0.6617222183391614,
 0.6626160210996981,
 0.6624042307839172,
 0.6631680333855138,
 0.6635222518153464,
 0.6634431266883472,
 0.6634856748237966,
 0.6634478036618627,
 0.6626806163127901,
 0.6631708828198603,
 0.6630375381504079,
 0.6619038135365095,
 0.6619558320444557,
 0.6619850536893968,
 0.6619369280131314,
 0.66217246986407,
 0.6627156727695317,
 0.6618900229213813,
 0.6622812778772607,
 0.6617631963008194,
 0.662630812899018,
 0.6621699360743535,

cityscapes fog val adaptation 

In [27]:
results

[0.6486378098138081,
 0.6530596042522123,
 0.6576112123441219,
 0.6633468964125697,
 0.6656263392476721,
 0.6685528391158426,
 0.6703093663878957,
 0.6706799997150047,
 0.6713792122065719,
 0.6725736594067386,
 0.674734476306424,
 0.6753769006678368,
 0.6739544547369035,
 0.672518078301109,
 0.6712299118719689,
 0.6708664823102284,
 0.6706907931373409,
 0.6683583435331937,
 0.66881773288257,
 0.6684461378036467,
 0.6671762151013129,
 0.6669452634442662,
 0.6668210869496118,
 0.6669318618623119,
 0.666594795134239,
 0.6665391451169185,
 0.6657532383180694,
 0.6676761272713531,
 0.6695559450151048,
 0.6697867667659019,
 0.6694050545500523,
 0.6695418877135415,
 0.669589545452666,
 0.6694326970759651,
 0.6690622224238202,
 0.6687263499808138,
 0.668206690657578,
 0.6681996742505921,
 0.6673825201021062,
 0.6661901478642002,
 0.6660458281427186,
 0.6659199404942046,
 0.6656890858231741,
 0.6650084398886302,
 0.6647549172735209,
 0.664146328973904,
 0.6637961437586765,
 0.6631078043064502,


cityscapes rain adaption on train

In [17]:
results

[0.5185679770823309,
 0.5363141317460808,
 0.5485649914052527,
 0.5622206638771065,
 0.5730894297622422,
 0.5807707596549841,
 0.5829549004310819,
 0.5877890852790915,
 0.5902769064180536,
 0.5921669408678931,
 0.5907398611326619,
 0.5891390073841061,
 0.5894710587892886,
 0.59198079882937,
 0.5895271909907736,
 0.5892144783554818,
 0.5898702861611402,
 0.5904537163995373,
 0.593093966219925,
 0.5944125620477667,
 0.5924851927908139,
 0.5936723806051762,
 0.5911122175228827,
 0.5912613638872654,
 0.5909239342017238,
 0.5924613993911074,
 0.5913890713249317,
 0.5920704969128471,
 0.5923071850977177,
 0.5925113606991549,
 0.5924013367369101,
 0.5924109399221709,
 0.5928552793128959,
 0.5939134640014533,
 0.5937671477347909,
 0.5943682654253786,
 0.5949574997715835,
 0.5954948833731919,
 0.5963222343797208,
 0.5961380923703438,
 0.5968397452377894,
 0.596302307328578,
 0.596837115252294,
 0.5971633382508429,
 0.5967905368579102,
 0.5973678122896272,
 0.5976450956512468,
 0.597601941619612

cityscapes rain, adaptation and validation on val set

In [19]:
results

[0.524217176131537,
 0.534658630625097,
 0.5440443861981522,
 0.5571952506309825,
 0.5649325150188156,
 0.5684358540394493,
 0.570375291655611,
 0.5763165885936994,
 0.5795915540730254,
 0.5785096066144567,
 0.5802279133751775,
 0.5841991186434793,
 0.5853719429885222,
 0.5854308727771932,
 0.5837749469502078,
 0.5846104881071712,
 0.5847644300414407,
 0.5877581918012259,
 0.5865059797935943,
 0.5863870630718757,
 0.5870547573787653,
 0.5870645178710666,
 0.5883612569552631,
 0.5878617447301149,
 0.589108355065346,
 0.5887846265190301,
 0.5900899478939985,
 0.5898560778351346,
 0.5886320300311705,
 0.5902838706236736,
 0.5884415268629298,
 0.5883900425275291,
 0.588960159980166,
 0.589360298700421,
 0.5902800024644529,
 0.5912114537568469,
 0.5908985337083829,
 0.591119539093099,
 0.591303356376169,
 0.592063229669057,
 0.5921843079463297,
 0.5915486373483885,
 0.591862580858852,
 0.5924866541200096,
 0.5926003581137875,
 0.5924160613090845,
 0.5927139751797489,
 0.5933570211738745,
 0

DEBUG

In [8]:
import utils
import numpy as np
from PIL import Image
denorm = utils.Denormalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225])

In [18]:
image, _ = next(iter(cityscapes_train_loader))
inputs = get_adaption_inputs(image, tr_transform_adapt, device)
for i in range(len(inputs)):
        image = inputs[i].detach().cpu().numpy()

        image = (denorm(image) * 255).transpose(1, 2, 0).astype(np.uint8)
        Image.fromarray(image).save('results/test_image_{}.png'.format(i))

NameError: name 'cityscapes_train_loader' is not defined

In [12]:
validation_set = datasets.acdc.Acdc(r"/vol/tmp/phanfran/acdc/", 'val', 'semantic',transform=val_transform, weather=WEATHER)
BATCH_SIZE = 3
val_loader = data.DataLoader(
        validation_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=8,
        drop_last=True)

In [9]:
cityscapes_val = Cityscapes(root='/vol/tmp/phanfran/cityscapes/',
                             subset=WEATHER,
                             split='val',
                             transform=val_transform)

BATCH_SIZE = 5
cityscapes_val_loader = data.DataLoader(
        cityscapes_val, batch_size=BATCH_SIZE, shuffle=False, num_workers=2,
        drop_last=True)

In [10]:
images, targets = next(iter(cityscapes_val_loader))

In [11]:
model.eval()
image = images.to(device, dtype=torch.float32)
labels = targets.to(device, dtype=torch.long)
output = model(image)

preds = output['out'].max(dim=1)[1].cpu().numpy()
targets = labels.cpu().numpy()

for i in range(len(images)):
    image = images[i].detach().cpu().numpy()
    target = targets[i]
    pred = preds[i]

    image = (denorm(image) * 255).transpose(1, 2, 0).astype(np.uint8)
    decode_target = cityscapes_val_loader.dataset.decode_target(target).astype(np.uint8)
    decode_pred = cityscapes_val_loader.dataset.decode_target(pred).astype(np.uint8)

    Image.fromarray(image).save('results/test_image_CS_fog_{}.png'.format(i))
    Image.fromarray(decode_target).save('results/test_target_CS_fog_{}.png'.format(i))
    Image.fromarray(decode_pred).save('results/test_pred_CS_fog_adapted_{}.png'.format(i))